# MovieLens Dataset


Based on the Keras code at: https://www.kaggle.com/colinmorris/embedding-layers

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import random

import tensorflow as tf
import jax
import jax.numpy as jnp
import functools
import optax
from flax import struct
from flax import linen as nn
from flax.training import train_state

In [3]:
ratings_df = pd.read_csv("data/movie-lens/rating.csv", usecols=['userId', 'movieId', 'rating'])
movies_df = pd.read_csv("data/movie-lens/movie.csv", usecols=['movieId', 'title'])
df = ratings_df.merge(movies_df, on='movieId').sort_values(by='userId')

In [4]:
# Normalize the ratings
df['y'] = (df['rating'] - df['rating'].mean())/df['rating'].std()
df

,userId,movieId,rating,title,y
0,1,2,3.5,Jumanji (1995),-0.024267
505014,1,541,4.0,Blade Runner (1982),0.451023
2380423,1,6807,3.5,Monty Python's The Meaning of Life (1983),-0.024267
2378699,1,6774,4.0,Videodrome (1983),0.451023
2376750,1,6755,3.5,Bubba Ho-tep (2002),-0.024267
...,...,...,...,...,...
8765330,138493,4343,4.0,Evolution (2001),0.451023
2266440,138493,5679,5.0,"Ring, The (2002)",1.401604
17434898,138493,4673,4.0,Tango & Cash (1989),0.451023
14982572,138493,836,3.5,Chain Reaction (1996),-0.024267


In [5]:
# Shuffle
df = df.sample(frac=1, random_state=1) 
# First 10,000 for test
test_df = df.iloc[:10000]
# Rest for train
train_df = df.iloc[10000:]

In [6]:
n_movies = len(df.movieId.unique())
n_users = len(df.userId.unique())
print(
    "{1:,} distinct users rated {0:,} different movies (total ratings = {2:,})".format(
        n_movies, n_users, len(df),
    )
)

138,493 distinct users rated 26,744 different movies (total ratings = 20,000,263)


In [7]:
@struct.dataclass
class MovieLensConfig:
  """Global hyperparameters for our MovieLens Model"""
  users_size: int = n_users   # 138,493
  movies_size: int = n_movies # 26,744
  emb_dim: int = 8
  dense_size_0: int = 32
  dense_size_1: int = 4
  out_size: int = 1
  num_epochs: int = 40
  batch_size: int = 8192
  lr: float = 0.0015
  momentum: float = 0.1

In [8]:
class MovieLensModel(nn.Module):
  """A simple embedding model."""

  config: MovieLensConfig

  @nn.compact
  def __call__(self, user_id, movie_id):
    
    cfg = self.config
    
    user_id = user_id.astype('int32')
    user_emb = nn.Embed(num_embeddings=cfg.users_size, features=cfg.emb_dim, name='user')(user_id)
    
    movie_id = movie_id.astype('int32')
    movie_emb = nn.Embed(num_embeddings=cfg.movies_size, features=cfg.emb_dim, name='movie')(movie_id)
    
    x = jnp.concatenate((user_emb, movie_emb))
    
    x = nn.Dense(cfg.dense_size_0, kernel_init=nn.initializers.xavier_uniform(), bias_init=nn.initializers.normal(stddev=1e-6))(x)
    x = nn.relu(x)
    x = nn.Dense(cfg.dense_size_1, kernel_init=nn.initializers.xavier_uniform(), bias_init=nn.initializers.normal(stddev=1e-6))(x)
    x = nn.relu(x)
    x = nn.Dense(cfg.out_size, kernel_init=nn.initializers.xavier_uniform(), bias_init=nn.initializers.normal(stddev=1e-6))(x)

    return x

In [9]:
@functools.partial(jax.jit, static_argnums=(0))
def apply_model(cfg, state, user_id, movie_id, rating):
  
  def loss_fn(params):
    logits = MovieLensModel(cfg).apply({'params': params}, user_id, movie_id)
    loss = jnp.mean(optax.l2_loss(predictions=logits, targets=rating))
    return loss, logits
  
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)
  return grads, loss    

In [10]:
@jax.jit
def update_model(state, grads):
  return state.apply_gradients(grads=grads)

In [11]:
def train_epoch(cfg, state, train_df, rng):
  """Train for a single epoch."""
  train_df_size = len(train_df)
  steps_per_epoch = train_df_size // cfg.batch_size
  
  perms = jax.random.permutation(rng, train_df_size)
  perms = perms[:steps_per_epoch * cfg.batch_size]  # skip incomplete batch
  perms = perms.reshape((steps_per_epoch, cfg.batch_size))
  
  epoch_loss = []
  
  for perm in perms:
    # x
    batch_user_id = train_df.iloc[perm]['userId'].values
    batch_movie_id = train_df.iloc[perm]['movieId'].values
    # y
    batch_rating = train_df.iloc[perm]['y'].values
    
    grads, loss = apply_model(cfg, state, batch_user_id, batch_movie_id, batch_rating)
    state = update_model(state, grads)
    epoch_loss.append(loss)
    
  train_loss = np.mean(epoch_loss)
  return state, train_loss

In [12]:
def create_train_state(rng, config):
  """Creates initial `TrainState`."""
  model = MovieLensModel(config)
  user_id_fake = jnp.zeros((1), jnp.int32)
  movie_id_fake = jnp.zeros((1), jnp.int32)
  # Pass fake values through our model to initialize the parameters
  params = model.init(rng, user_id_fake, movie_id_fake)['params']
  
  # TODO(joshvarty): Consider other optimizers.
  tx = optax.sgd(config.lr)
  return train_state.TrainState.create(
      apply_fn=model.apply, params=params, tx=tx)

In [13]:
# Check if we can create trainstate
config = MovieLensConfig()
rng = jax.random.PRNGKey(0)
rng, init_rng = jax.random.split(rng)
state = create_train_state(init_rng, config)
print("Success.")

Success.


In [14]:
def train_and_evaluate(config, train_df, test_df):
  rng = jax.random.PRNGKey(0)
  rng, init_rng = jax.random.split(rng)
  state = create_train_state(init_rng, config)
  
  for epoch in range(1, config.num_epochs + 1):
    rng, input_rng = jax.random.split(rng)
    state, train_loss = train_epoch(config, state, train_df, rng)
    
    user_id = test_df['userId'].values
    movie_id = test_df['movieId'].values
    rating = test_df['y'].values
    _, test_loss = apply_model(config, state, user_id, movie_id, rating)
    
    print(
        'epoch:% 3d, train_loss: %.4f, test_loss: %.4f'
        % (epoch, train_loss, test_loss))
    
  return state

In [15]:
# Sample output
user_id = test_df['userId'].values
movie_id = test_df['movieId'].values
rating = test_df['y'].values
logits = MovieLensModel(config).apply({'params': state.params}, user_id, movie_id)
logits

DeviceArray([[-0.24850503],
             [-0.12539954],
             [-0.1477067 ],
             ...,
             [ 0.10562674],
             [-0.10492511],
             [ 0.16549747]], dtype=float32)

In [16]:
# Hide any GPUs from TensorFlow. Otherwise TF might reserve memory and make
# it unavailable to JAX.
tf.config.experimental.set_visible_devices([], 'GPU')
config = MovieLensConfig()

state = train_and_evaluate(config, train_df, test_df)

epoch:  1, train_loss: 0.5048, test_loss: 0.5157
epoch:  2, train_loss: 0.5026, test_loss: 0.5144
epoch:  3, train_loss: 0.5016, test_loss: 0.5137
epoch:  4, train_loss: 0.5011, test_loss: 0.5134
epoch:  5, train_loss: 0.5008, test_loss: 0.5132
epoch:  6, train_loss: 0.5007, test_loss: 0.5130
epoch:  7, train_loss: 0.5006, test_loss: 0.5129
epoch:  8, train_loss: 0.5005, test_loss: 0.5129
epoch:  9, train_loss: 0.5004, test_loss: 0.5128
epoch: 10, train_loss: 0.5004, test_loss: 0.5128
epoch: 11, train_loss: 0.5003, test_loss: 0.5127
epoch: 12, train_loss: 0.5003, test_loss: 0.5127
epoch: 13, train_loss: 0.5003, test_loss: 0.5127
epoch: 14, train_loss: 0.5002, test_loss: 0.5126
epoch: 15, train_loss: 0.5002, test_loss: 0.5126
epoch: 16, train_loss: 0.5002, test_loss: 0.5126
epoch: 17, train_loss: 0.5002, test_loss: 0.5126
epoch: 18, train_loss: 0.5002, test_loss: 0.5126
epoch: 19, train_loss: 0.5001, test_loss: 0.5126
epoch: 20, train_loss: 0.5001, test_loss: 0.5126
epoch: 21, train_los

In [17]:
# Sample output
user_id = test_df['userId'].values
movie_id = test_df['movieId'].values
rating = test_df['y'].values
logits = MovieLensModel(config).apply({'params': state.params}, user_id, movie_id)
logits

DeviceArray([[-0.02975536],
             [ 0.00179531],
             [-0.01827695],
             ...,
             [ 0.00573348],
             [-0.00702804],
             [ 0.00250866]], dtype=float32)

## Thoughts

It's not working:

 - Are the ratings balanced? Can we rebalance?
 - Are the User/Movie embedding updating?
 - Do we want different LRs for the embeddings vs weights?


###  What is the best train/test loss we've seen? 

In [18]:
# train loss: 0.5051
# test loss:  0.5174
@struct.dataclass
class MovieLensConfig:
  """Global hyperparameters for our MovieLens Model"""
  users_size: int = n_users   # 138,493
  movies_size: int = n_movies # 26,744
  emb_dim: int = 8
  dense_size_0: int = 32
  dense_size_1: int = 4
  out_size: int = 1
  num_epochs: int = 40
  batch_size: int = 8192
  lr: float = 0.000015
  momentum: float = 0.1
    
    
# train loss: 0.5010
# test loss:  0.5134
@struct.dataclass
class MovieLensConfig:
  """Global hyperparameters for our MovieLens Model"""
  users_size: int = n_users   # 138,493
  movies_size: int = n_movies # 26,744
  emb_dim: int = 8
  dense_size_0: int = 32
  dense_size_1: int = 4
  out_size: int = 1
  num_epochs: int = 40
  batch_size: int = 8192
  lr: float = 0.00015
  momentum: float = 0.1
    
    
# train loss: 0.5000
# test_loss:  0.5125
@struct.dataclass
class MovieLensConfig:
  """Global hyperparameters for our MovieLens Model"""
  users_size: int = n_users   # 138,493
  movies_size: int = n_movies # 26,744
  emb_dim: int = 8
  dense_size_0: int = 32
  dense_size_1: int = 4
  out_size: int = 1
  num_epochs: int = 40
  batch_size: int = 8192
  lr: float = 0.0015
  momentum: float = 0.1


## TODO: Is this working?

- Add a metric. Maybe Mean Absolute Error?
- Compare MAE from predicting mean to MAE from our model.
- Look at a user's ratings compared to the predicted ratings? Are they reasonable?
- Find movies that are similar to one another.